In [ ]:
!pip3 install osmnx
!pip3 install geojson
!pip3 install shapely.constructive
import pandas as pd
import numpy as np
import csv
import plotly.express as px
import osmnx as ox
import geopandas as gpd
import json
import requests
# import regex as re
%matplotlib inline
from shapely.geometry import Point
from geopandas import datasets, GeoDataFrame, read_file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# workdir = '/content/drive/MyDrive/Colab Notebooks/Hazus Data/' # for Tina
workdir = '/content/drive/MyDrive/Hazus Data/'                 # for Adam

# 1 Importing tables
#### The file path is my own file path in the google drive folder. Other people might need to change according to their file path locations in their google drive.

In [ ]:
# A function to convert csv file to df

def read_tables(path,state):
  df = pd.read_csv(path,sep =',')
  df['State'] = state
  cols = list(df)
  cols = [cols[-1]] + cols[:-1]
  cols = df[cols]
  return cols

In [ ]:
local_path_CA_B = '/Hazus_data/hzBldgCountOccupB_hzBldgCountOccuT/CA_hzBldgCountOccupB_table.csv'
path_CA_B = f'{workdir}hz_B_T/CA_hzBldgCountOccupB_table.csv'
state = 'CA'
CA_hzBldgCountOccupB = read_tables(path_CA_B,state)
CA_hzBldgCountOccupB

In [ ]:
path_CA_T = f'{workdir}hz_B_T/CA_hzBldgCountOccupT_table.csv'
state = 'CA'
CA_hzBldgCountOccupT = read_tables(path_CA_T,state)
CA_hzBldgCountOccupT

In [ ]:
path_OR_B = '/content/drive/MyDrive/Colab Notebooks/Hazus Data/hz_B_T/OR_hzBldgCountOccupB_table.csv'
state = 'OR'
OR_hzBldgCountOccupB = read_tables(path_OR_B,state)
OR_hzBldgCountOccupB

In [ ]:
path_OR_T = '/content/drive/MyDrive/Colab Notebooks/Hazus Data/hz_B_T/OR_hzBldgCountOccupT_table.csv'
state = 'OR'
OR_hzBldgCountOccupT = read_tables(path_OR_T,state)
OR_hzBldgCountOccupT

In [ ]:
path_WA_B = '/content/drive/MyDrive/Colab Notebooks/Hazus Data/hz_B_T/WA_hzBldgCountOccupB_table.csv'
state = 'WA'
WA_hzBldgCountOccupB = read_tables(path_WA_B,state)
WA_hzBldgCountOccupB

In [ ]:
path_WA_T = '/content/drive/MyDrive/Colab Notebooks/Hazus Data/hz_B_T/WA_hzBldgCountOccupt_table.csv'
state = 'WA'
WA_hzBldgCountOccupT = read_tables(path_WA_T,state)
WA_hzBldgCountOccupT

In [ ]:
hzBldgCountOccupB_tables = CA_hzBldgCountOccupB.append(OR_hzBldgCountOccupB).append(WA_hzBldgCountOccupB)
hzBldgCountOccupB_tables

In [ ]:
B_len_check = len(CA_hzBldgCountOccupB) + len(OR_hzBldgCountOccupB) + len(WA_hzBldgCountOccupB) == len(hzBldgCountOccupB_tables)
B_len_check

In [ ]:
hzBldgCountOccupT_tables = CA_hzBldgCountOccupT.append(OR_hzBldgCountOccupT).append(WA_hzBldgCountOccupT)
hzBldgCountOccupT_tables

In [ ]:
T_len_check = len(CA_hzBldgCountOccupT) + len(OR_hzBldgCountOccupT) + len(WA_hzBldgCountOccupT) == len(hzBldgCountOccupT_tables)
T_len_check

# 2 Find Sum of the buildings in hazus data in CA

In [ ]:
CA_hzBldgCountOccupB_sum = CA_hzBldgCountOccupB
CA_hzBldgCountOccupB_sum['total'] = CA_hzBldgCountOccupB.loc[:,'RES1I':'EDU2I'].sum(axis=1)
CA_hzBldgCountOccupB_sum['total'] = CA_hzBldgCountOccupB.loc[:,'RES1I':'EDU2I'].sum(axis=1)
CA_hzBldgCountOccupB_sum["RES3I_total"] = CA_hzBldgCountOccupB_sum.loc[:,"RES3AI":"RES3FI"].sum(axis=1)
CA_hzBldgCountOccupB_sum = CA_hzBldgCountOccupB_sum[["CensusBlock","RES1I","RES3I_total","total"]]
CA_hzBldgCountOccupB_sum

# 3 Import census block polygon

This is the polygon in alameda county in CA state

In [ ]:
ST_poly = gpd.read_file(f'{workdir}tabblock2010_06_pophu.zip')
ST_poly

In [ ]:
ALAMEDA_HOUSES = ST_poly.loc[ST_poly['COUNTYFP10']== "001"]
ALAMEDA_HOUSES = ALAMEDA_HOUSES.to_crs(4326)
# ALAMEDA_HOUSES = ALAMEDA_HOUSES.loc[ALAMEDA_HOUSES['HOUSING10'] > 0]
ALAMEDA_HOUSES["BLOCKID10"] = ALAMEDA_HOUSES["BLOCKID10"].astype(int)
ALAMEDA_HOUSES

In [ ]:
CT_poly = gpd.read_file(f'{workdir}CA_ALAMEDA_2010.zip')
CT_poly

In [ ]:
CT_poly["GEOID10"] = CT_poly["GEOID10"].astype(str).astype(int)

Join hazus CA census block table with the census block table from tiger. The next step probably will convert the lat and lon into polygons to compare with footprint.

In [ ]:
join_table = CA_hzBldgCountOccupB_sum.merge(CT_poly, how = 'left',left_on = 'CensusBlock',right_on = 'GEOID10')
join_table.dropna()
join_table_gpd = gpd.GeoDataFrame(join_table, geometry='geometry', crs=4326)
join_table_gpd = join_table_gpd.dropna()
join_table_gpd

Convert the dataframe to Geopandas Dataframe.
Since plotly can't process large datasets, I select the top 1000 row to see if plotly in function normally.

In [ ]:
res_plt = join_table_gpd[["CensusBlock","total","GEOID10","geometry","RES1I","RES3I_total"]]
res_plt

In [ ]:
housing_diff = res_plt.merge(ALAMEDA_HOUSES, how = 'inner', left_on = "CensusBlock",right_on = "BLOCKID10" )
housing_diff = housing_diff[["CensusBlock","total","geometry_x","HOUSING10","RES1I","RES3I_total"]]
housing_diff["difference"] = housing_diff["HOUSING10"] - housing_diff["total"]
housing_diff_gpd = gpd.GeoDataFrame(housing_diff, geometry='geometry_x', crs=4326)
housing_diff_gpd

This is the hazus data sum score graph for Alameda county

In [ ]:
place_name = 'Emeryville, California'
footprints = ox.geometries_from_place(place_name, tags = {'building': True})
footprints.index = np.arange(footprints.shape[0])
footprints

In [ ]:
footprints.columns

In [ ]:
footprints["area"] = footprints.to_crs({'proj':'cea'}).area
footprints = footprints.loc[footprints["area"] >= 50]
footprints

In [ ]:
fig = px.choropleth_mapbox(
    footprints,
    geojson=footprints.geometry,
    locations=footprints.index,
    center={"lat": 37.8384729, "lon": -122.3016820},
    mapbox_style="carto-positron",
    color_continuous_scale='Portland',
    opacity=0.9,
    zoom=13,
    height=800,
    width=1000,
    title = "Emeryville Footprints"
  )
fig.show()

In [ ]:
res_plt = res_plt.to_crs(4326)
fig = px.choropleth_mapbox(
    res_plt,
    geojson=res_plt.geometry,
    locations=res_plt.index,
    color = res_plt.index, #"total",
    center={"lat": 37.8384729, "lon": -122.3016820},
    mapbox_style="carto-positron",
    color_continuous_scale='Portland',
    opacity=0.9,
    zoom=13,
    height=800,
    width=1000,
    title = "Alameda County Hazus Score graph"
  )
fig.show()

This is the Alameda county housing units in 2010.

In [ ]:
ALAMEDA_HOUSES.index = np.arange(ALAMEDA_HOUSES.shape[0])
fig = px.choropleth_mapbox(
    ALAMEDA_HOUSES,
    geojson=ALAMEDA_HOUSES.geometry,
    locations=ALAMEDA_HOUSES.index,
    color="HOUSING10",
    center={"lat": 37.8, "lon": -122.27},
    mapbox_style="carto-positron",
    color_continuous_scale='Portland',
    opacity=0.9,
    zoom=11,
    height=800,
    range_color = (0,500),
    width=1000,
    title = "Alameda County Housing units graph"
  )
# fig.update_yaxes(range=[0,500])
fig.show()

In [ ]:
housing_diff_gpd.index = np.arange(housing_diff_gpd.shape[0])
fig = px.choropleth_mapbox(
    housing_diff_gpd,
    geojson=housing_diff_gpd.geometry_x,
    locations=housing_diff_gpd.index,
    color="difference",
    center={"lat": 37.8, "lon": -122.27},
    mapbox_style="carto-positron",
    color_continuous_scale='Portland',
    opacity=0.9,
    zoom=11,
    range_color = (-50,500),
    height=800,
    width=1000,
    title ="The difference between Hazus score and Housing units graph in Alameda County"
  )
fig.show()

# 4 Emeryville maps

In [ ]:
emer_hazus = res_plt.to_crs(4326).sjoin(footprints, how = "inner")
emer_hazus

In [ ]:
fwc = footprints.copy()
# we play around with the geometry a bit here
# store the polygons in another column
fwc['geometry_polygon'] = fwc['geometry'].copy()
# load their centroids as the geometry
fwc['geometry'] = fwc['geometry'].centroid
# use the centroids in the sjoin to have a unique footprint - census block assignment
fp_hazus = fwc.sjoin(res_plt.to_crs(4326), how = "inner")
# save the centroids in another column
fp_hazus['geometry_centroid'] = fp_hazus['geometry'].copy()
# load back the polygons for future work
fp_hazus['geometry'] = fp_hazus['geometry_polygon']
fp_hazus

In [ ]:
#emer_hazus.index = np.arange(emer_hazus.shape[0])

FP_polygons = fp_hazus.copy()

# take the unique Census Block indexes from the footprint dataframe (long, unordered numbers)
CB_ids = FP_polygons['index_right'].unique()
# and shuffle them (to avoid having nearby census blocks next to each other in the list)
np.random.shuffle(CB_ids)

# then use the list of indexes to get the census block polygons from res_plt
CB_polygons = res_plt.loc[CB_ids].copy()

# and add a CID column with an ordered list of ids (we'll use this for coloring)
CB_polygons['CID'] = np.arange(CB_polygons.shape[0])
# also add an opacity column to have semi-transparent block polygons
CB_polygons['opacity'] = 0.25

# # now count the number of footprints in each census block
FP_polygons['count'] = 1
FP_count = FP_polygons.groupby('index_right').sum()
# and store the footprint count in the CB dataframe that we've just prepared
CB_polygons.loc[FP_count.index, 'footprint_count'] = FP_count['count']

# finally, store the unique ID of the census block for each footprint too
# (this ensures that we'll give the footprint polygons the right color)
for unique_id, original_id in enumerate(CB_ids):
  FP_polygons.loc[FP_polygons['index_right']==original_id, 'CID'] = unique_id

FP_polygons['CID'] = FP_polygons['CID'].astype(int)
# set footprint opacity to 1.0 - i.e., non-transparent
FP_polygons['opacity'] = 1.0

# concatenate the FP and CB dataframes so that we can plot the shapes together
ch_plt = pd.concat([FP_polygons, CB_polygons], axis=0)

#ch_plt = ch_plt.iloc[:100]
fig = px.choropleth_mapbox(
    ch_plt,
    geojson=ch_plt.geometry,
    locations=ch_plt.index,
    color="CID",
    hover_name='footprint_count',
    center={"lat": 37.8, "lon": -122.27},
    mapbox_style="carto-positron",
    color_continuous_scale='Rainbow',
    opacity=ch_plt['opacity'],
    zoom=11,
    range_color = (-50,100),
    height=800,
    width=1000,
    title = "Emeryville Hazus Score graph"
  )
fig.show()

In [ ]:
CB_polygons["HZ_diff"] = CB_polygons["total"]-CB_polygons["footprint_count"]
CB_polygons["level"] = pd.cut(x=CB_polygons["HZ_diff"], bins=[-100, -50,-0.01,0.99, 10, 20,50, 200],
                    labels=['<-50','<0', '0','0 to 10',
                            '10 to 20',">20",">50"])
CB_polygons_dropna = CB_polygons.dropna(subset = ["HZ_diff","footprint_count"])
CB_polygons_dropna["opacity"] = 0.25
CB_polygons_dropna = CB_polygons_dropna[["geometry","CensusBlock","total","CID","footprint_count","HZ_diff","level","opacity","RES1I","RES3I_total"]]
CB_polygons_dropna

In [ ]:
FP_polygons['HZ_diff'] = 0
FP_polygons['level'] = '0'
FP_polygons["RES3I_total"] = 0
FP_polygons["RES1I"] = 0
ch_plt = pd.concat([FP_polygons, CB_polygons_dropna], axis=0)

fig = px.choropleth_mapbox(
    ch_plt,
    geojson=ch_plt.geometry,
    locations=ch_plt.index,
    color="HZ_diff", #HZ_diff",
    hover_name="level",
    center={"lat": 37.8, "lon": -122.27},
    mapbox_style="carto-positron",
    color_continuous_scale='Rainbow',
    opacity=ch_plt['opacity'],
    zoom=11,
    range_color = (-50,100),
    height=800,
    width=1000,
    title = "Emeryville Hazus Score difference graph"
  )
fig.show()

In [ ]:
fig = px.choropleth_mapbox(
    ch_plt,
    geojson=ch_plt.geometry,
    locations=ch_plt.index,
    color="RES1I", #HZ_diff",
    hover_name="footprint_count",
    center={"lat": 37.8, "lon": -122.27},
    mapbox_style="carto-positron",
    color_continuous_scale='Rainbow',
    opacity=ch_plt['opacity'],
    zoom=11,
    range_color = (-50,100),
    height=800,
    width=1000,
    title = "Emeryville Hazus RES1I graph"
  )
fig.show()

In [ ]:
fig = px.choropleth_mapbox(
    ch_plt,
    geojson=ch_plt.geometry,
    locations=ch_plt.index,
    color="RES3I_total", #HZ_diff",
    hover_name="footprint_count",
    center={"lat": 37.8, "lon": -122.27},
    mapbox_style="carto-positron",
    color_continuous_scale='Rainbow',
    opacity=ch_plt['opacity'],
    zoom=11,
    range_color = (-50,100),
    height=800,
    width=1000,
    title = "Emeryville Hazus RES3I_total graph"
  )
fig.show()

### Sanity check

In [ ]:
ch_plt.loc[8937] # small building check == true

In [ ]:
ch_plt.loc[8930]

In [ ]:
ch_plt.loc[8829] # previous census block check == true

In [ ]:
ch_plt.loc[8960] # overlap census block check == false buildings repeatedly counted

In [ ]:
ch_plt.loc[8959] # overlap census block check == false

In [ ]:
ch_plt.loc[8818] # overlap census block check == false


In [ ]:
ch_plt.loc[ch_plt["index_right"] == 8819] # footprint counts is correct but one of the buildings is so small


## Housing Units graph

In [ ]:
display(ALAMEDA_HOUSES)
display(emer_hazus)

In [ ]:
block_id = ch_plt["CensusBlock"]

In [ ]:
emer_diff_on_levels = ALAMEDA_HOUSES.sjoin(res_plt.to_crs(4326), how = "inner", predicate = "contains")
emer_diff_on_levels.head(10)

In [ ]:
HS_diff = emer_diff_on_levels.loc[emer_diff_on_levels["CensusBlock"].isin(block_id)]
HS_diff["HS_diff"] = HS_diff["HOUSING10"] - HS_diff["total"]
HS_diff['geometry_polygon'] = HS_diff['geometry'].copy()
# load their centroids as the geometry
HS_diff['geometry'] = HS_diff['geometry'].centroid
# use the centroids in the sjoin to have a unique footprint - census block assignment
# fp_hazus = fwc.sjoin(res_plt.to_crs(4326), how = "inner")
# save the centroids in another column
HS_diff['geometry_centroid'] = HS_diff['geometry'].copy()
# load back the polygons for future work
HS_diff['geometry'] = HS_diff['geometry_polygon']
HS_diff.to_crs(4326)
HS_diff = HS_diff[["HOUSING10","geometry","index_right","CensusBlock","total","HS_diff","geometry_polygon","geometry_centroid"]]
HS_diff["opacity"] = 0.25
HS_diff

In [ ]:
HS_diff = HS_diff.sort_values(by = "HS_diff")
HS_diff

HS_diff["level"] = pd.cut(x=HS_diff['HS_diff'], bins=[-100, -50,-0.01,0.99, 10, 20,50, 1000],
                    labels=['<-50','<0', '0','0 to 10',
                            '10 to 20',">20",">50"])
HS_diff.index = HS_diff["index_right"]
fig = px.choropleth_mapbox(
    HS_diff,
    geojson=HS_diff.geometry,
    locations=HS_diff.index,
    color="HS_diff",
    hover_name="level",
    center={"lat": 37.8, "lon": -122.27},
    mapbox_style="carto-positron",
    color_continuous_scale='Rainbow',
    opacity=HS_diff['opacity'],
    zoom=11,
    range_color = (-50,100),
    height=800,
    width=1000,
    title = "Emeryville Hazus Score difference graph"
  )
fig.show()

In [ ]:
HZ_HS_diff = HS_diff.copy()
HZ_HS_diff = HS_diff.merge(ch_plt, on = "CensusBlock")
HZ_HS_diff["total_diff"] = abs(HZ_HS_diff["HS_diff"]) - abs(HZ_HS_diff["HZ_diff"])
HZ_HS_diff = gpd.GeoDataFrame(HZ_HS_diff,geometry="geometry_x")


In [ ]:
HZ_HS_diff.index = HZ_HS_diff["index_right"]
fig = px.choropleth_mapbox(
    HZ_HS_diff,
    geojson=HZ_HS_diff.geometry_x,
    locations=HZ_HS_diff.index,
    color="total_diff",
    hover_name="level_x",
    center={"lat": 37.8, "lon": -122.27},
    mapbox_style="carto-positron",
    color_continuous_scale='Rainbow',
    opacity=HS_diff['opacity'],
    zoom=11,
    range_color = (-50,100),
    height=800,
    width=1000,
    title = "Emeryville Hazus Score difference graph"
  )
fig.show()

# Brookyln Area Check

In [ ]:
# Read dataframe from Hazus
path_CA_B = f'{workdir}NY_hzBldgCountOccupB.csv'
state = 'NY'
NY_hzBldgCountOccupB = read_tables(path_CA_B,state)
# Sum the total score for NY
NY_hzBldgCountOccupB_sum = NY_hzBldgCountOccupB.copy()
NY_hzBldgCountOccupB_sum['total'] = NY_hzBldgCountOccupB.loc[:,'RES1I':'EDU2I'].sum(axis=1)
NY_hzBldgCountOccupB_sum['total_RES'] = NY_hzBldgCountOccupB.loc[:,'RES1I':'RES3FI'].sum(axis=1)
NY_hzBldgCountOccupB_sum = NY_hzBldgCountOccupB_sum[["CensusBlock","RES1I","RES3AI","RES3BI","RES3CI","RES3DI","RES3EI","RES3FI","total","total_RES"]]
NY_hzBldgCountOccupB_sum["RES3I_total"] = NY_hzBldgCountOccupB_sum.loc[:,"RES3AI":"RES3FI"].sum(axis=1)
NY_hzBldgCountOccupB_sum = NY_hzBldgCountOccupB_sum[["CensusBlock","RES1I","RES3I_total","total","total_RES"]]
NY_hzBldgCountOccupB_sum

In [ ]:
NY_hzBldgCountOccupB.columns

In [ ]:
place_name = "Park Slope, Kings County, New York, United States"
NY_footprints = ox.geometries_from_place(place_name, tags = {'building': True})
NY_footprints.index = np.arange(NY_footprints.shape[0])
NY_footprints

In [ ]:
NY_footprints = NY_footprints.copy()
NY_footprints["area"] = NY_footprints["geometry"].to_crs({'proj':'cea'}).area
NY_footprints = NY_footprints.loc[NY_footprints["area"] >= 50]
NY_footprints

In [ ]:
fig = px.choropleth_mapbox(
    NY_footprints,
    geojson=NY_footprints.geometry,
    locations=NY_footprints.index,
    center={"lat": 40.67150, "lon": -73.97932},
    mapbox_style="carto-positron",
    color_continuous_scale='Portland',
    opacity=0.9,
    zoom=13,
    height=800,
    width=1000,
    title = "Park Slope Footprints"
  )
fig.show()

In [ ]:
NY_poly = gpd.read_file(f'{workdir}tl_2010_36047_tabblock10.zip')
NY_poly

In [ ]:
NY_poly["GEOID10"] = NY_poly["GEOID10"].astype(str).astype(int)
join_table = NY_hzBldgCountOccupB_sum.merge(NY_poly, how = "inner", left_on = "CensusBlock",right_on = "GEOID10")
join_table = join_table[["CensusBlock","RES1I","RES3I_total","total","total_RES","geometry","GEOID10"]]
PS_plt = join_table.copy()
PS_plt = gpd.GeoDataFrame(PS_plt, geometry = "geometry", crs = 4326)
PS_plt

In [ ]:
fwc = NY_footprints.copy()
# we play around with the geometry a bit here
# store the polygons in another column
fwc['geometry_polygon'] = fwc['geometry'].copy()
# load their centroids as the geometry
fwc['geometry'] = fwc['geometry'].centroid
# use the centroids in the sjoin to have a unique footprint - census block assignment
PS_hazus = fwc.sjoin(PS_plt.to_crs(4326), how = "inner")
# save the centroids in another column
PS_hazus['geometry_centroid'] = PS_hazus['geometry'].copy()
# load back the polygons for future work
PS_hazus['geometry'] = PS_hazus['geometry_polygon']

In [ ]:
SB_polygons = PS_hazus.copy()
CB_ids = SB_polygons["index_right"].unique()
np.random.shuffle(CB_ids)
CTB_polygons = PS_plt.loc[CB_ids].copy()

# take the unique Census Block indexes from the footprint dataframe (long, unordered numbers)
CB_ids = SB_polygons['index_right'].unique()
# and shuffle them (to avoid having nearby census blocks next to each other in the list)
np.random.shuffle(CB_ids)

# then use the list of indexes to get the census block polygons from res_plt
CTB_polygons = PS_plt.loc[CB_ids].copy()
# and add a CID column with an ordered list of ids (we'll use this for coloring)
CTB_polygons['CID'] = np.arange(CTB_polygons.shape[0])

# # also add an opacity column to have semi-transparent block polygons
CTB_polygons['opacity'] = 0.25
CTB_polygons
# # now count the number of footprints in each census block
SB_polygons['count'] = 1
FP_count = SB_polygons.groupby('index_right').sum()
FP_count
# # and store the footprint count in the CB dataframe that we've just prepared
CTB_polygons.loc[FP_count.index, 'footprint_count'] = FP_count['count']
CTB_polygons
# finally, store the unique ID of the census block for each footprint too
# (this ensures that we'll give the footprint polygons the right color)
for unique_id, original_id in enumerate(CB_ids):
  SB_polygons.loc[SB_polygons['index_right']==original_id, 'CID'] = unique_id

SB_polygons['CID'] = SB_polygons['CID'].astype(int)
# # set footprint opacity to 1.0 - i.e., non-transparent
SB_polygons['opacity'] = 1
SB_polygons
# # concatenate the FP and CB dataframes so that we can plot the shapes together
final_plt = pd.concat([SB_polygons, CTB_polygons], axis=0)
final_plt = final_plt.reset_index().drop(columns = "index")

In [ ]:
fig = px.choropleth_mapbox(
    final_plt,
    geojson=final_plt.geometry,
    locations=final_plt.index,
    color="CID",
    hover_name='footprint_count',
    center={"lat": 40.67105, "lon": -73.98099},
    mapbox_style="carto-positron",
    color_continuous_scale='Rainbow',
    opacity= final_plt['opacity'],
    zoom=13,
    range_color = (-50,100),
    height=800,
    width=1000,
    title = "Park Slope Hazus Score graph"
  )
fig.show()

In [ ]:
final_plt = final_plt[["geometry","geometry_polygon","index_right",'CensusBlock', 'total','total_RES', 'RES1I','RES3I_total','GEOID10','geometry_centroid', 'count', 'CID', 'opacity', 'footprint_count']]

In [ ]:
CTB_polygons["diff"] = CTB_polygons["RES1I"] + CTB_polygons["RES3I_total"] - CTB_polygons["footprint_count"]
CTB_polygons['ratio'] = CTB_polygons["RES1I"]/CTB_polygons["total_RES"]
CTB_polygons["level"] = pd.cut(x=CTB_polygons["diff"], bins=[-100, -50,-20,-0.01,0.99, 10, 20,50, 200],
                    labels=['<-50','<-20','<0', '0','0 to 10',
                            '10 to 20',">20",">50"])
CTB_polygons_dropna = CTB_polygons.dropna(subset = ["diff","footprint_count"])
CTB_polygons_dropna["opacity"] = 0.25
CTB_polygons_dropna = CTB_polygons_dropna[["geometry","CensusBlock","total","total_RES","CID","RES1I","RES3I_total","footprint_count","diff","level","opacity","ratio"]]
CTB_polygons_dropna['BG_ID'] = np.array([s[7:-3] for s in CTB_polygons_dropna['CensusBlock'].astype(str)]).astype(int)
for unique_id, BG_ID in enumerate(CTB_polygons_dropna['BG_ID'].unique()):
  CTB_polygons_dropna.loc[CTB_polygons_dropna['BG_ID']==BG_ID, 'BG_ID_simple'] = unique_id
CTB_polygons_dropna

In [ ]:
block_groups = CTB_polygons_dropna.copy()

block_groups = block_groups.groupby('BG_ID').sum().loc[:,['total_RES','footprint_count','diff']]

px.scatter(block_groups, x='total_RES', y='footprint_count', width=800, height=800)

In [ ]:
SB_polygons['diff'] = 0
SB_polygons['level'] = '0'
SB_polygons["opacity"] = 0.5
final_diff_plt = pd.concat([SB_polygons, CTB_polygons_dropna], axis=0)
final_diff_plt = final_diff_plt.reset_index().drop(columns = "index")
final_diff_plt

cmap = {
    '<-50': '#a50026',
    '<-20': '#d73027',
    '<0': '#f46d43',
    '0': '#ffffbf',
    '0 to 10': '#e0f3f8',
    '10 to 20': '#abd9e9',
    ">20": '#4575b4',
    ">50": '#313695'
}

fig = px.choropleth_mapbox(
    final_diff_plt,
    geojson=final_diff_plt.geometry,
    locations=final_diff_plt.index,
    color="level",
    #color="BG_ID_simple",
    #color = 'ratio',
    hover_name='level',
    hover_data=['RES1I','RES3I_total','footprint_count','diff'],
    center={"lat": 40.67105, "lon": -73.98099},
    mapbox_style="carto-positron",
    #color_continuous_scale='Rainbow',
    color_discrete_map= cmap,
    opacity= final_diff_plt['opacity'],
    zoom=13,
    #range_color = (-50,100),
    height=800,
    width=1000,
    title = "Park Slope Hazus Score diff graph"
  )
fig.show()

In [ ]:
SB_polygons["RES1I"] = 0
SB_polygons["RES3I_total"] = 0
RES_plt = pd.concat([SB_polygons,CTB_polygons_dropna], axis = 0)
RES_plt = RES_plt.reset_index().drop(columns = "index")
fig = px.choropleth_mapbox(
    RES_plt,
    geojson=RES_plt.geometry,
    locations=RES_plt.index,
    color="RES1I",
    hover_name='footprint_count',
    center={"lat": 40.67105, "lon": -73.98099},
    mapbox_style="carto-positron",
    color_continuous_scale='Rainbow',
    opacity= RES_plt['opacity'],
    zoom=13,
    range_color = (-50,100),
    height=800,
    width=1000,
    title = "Park Slope Hazus RES1I Score graph"
  )
fig.show()

In [ ]:
RES_plt = pd.concat([SB_polygons,CTB_polygons_dropna], axis = 0)
RES_plt = RES_plt.reset_index().drop(columns = "index")
fig = px.choropleth_mapbox(
    RES_plt,
    geojson=RES_plt.geometry,
    locations=RES_plt.index,
    color="RES3I_total",
    hover_name='footprint_count',
    hover_data=['RES1I','footprint_count'],
    center={"lat": 40.67105, "lon": -73.98099},
    mapbox_style="carto-positron",
    color_continuous_scale='Rainbow',
    opacity= RES_plt['opacity'],
    zoom=13,
    #range_color = (-50,100),
    height=800,
    width=1000,
    title = "Park Slope Hazus RES3I Score graph"
  )
fig.show()